In [ ]:
from larcv import larcv
larcv.load_pyutil
larcv.load_cvutil

import cv2
import ROOT
from ROOT import fcllite
from ROOT import geo2d
from ROOT import larocv

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# make a fake image...
a=np.zeros((485,485))

ctor = ROOT.std.vector(ROOT.cv.Point_('int'))()

ctor.clear()
    
pt_x=[]
pt_y=[]

ctor.push_back(ROOT.cv.Point_("int")(25,225))
ctor.push_back(ROOT.cv.Point_("int")(50,225))
ctor.push_back(ROOT.cv.Point_("int")(50,229))
ctor.push_back(ROOT.cv.Point_("int")(29,229))
ctor.push_back(ROOT.cv.Point_("int")(29,300))
ctor.push_back(ROOT.cv.Point_("int")(25,300))
ctor.push_back(ROOT.cv.Point_("int")(25,300))

ctor.push_back(ctor[0])
for pt in ctor:
    pt_x.append(pt.x)
    pt_y.append(pt.y)
    plt.plot(pt_x,pt_y,'o-',color='black',lw=1)

ax=plt.gca()
#ax.set_xlim(0,450)
#ax.set_ylim(0,450)
plt.grid()
plt.show()

# make a path representing this contour
path_list1 = []
for pt in ctor:
    path_list1.append((pt.x,pt.y)) 


In [ ]:
path1 = matplotlib.path.Path(path_list1)

img1 = path1.contains_points(np.row_stack(np.where(a==0)).T).reshape(485,485).astype(np.uint8)

img1=np.where(img1>0,img1+80,0)
img2 = img1.copy()
img3 = img1.copy()
M = cv2.getRotationMatrix2D((485/2,485/2),45,1)

#rotate 2
img2 = cv2.warpAffine(img2,M,(485,485))

#flip 2
img2 = img2[:,::-1]

#get nonzero for 2, and move it...
nzero=np.row_stack(np.where(img2>0)).T
img2[nzero[:,0],nzero[:,1]]=0
img2[nzero[:,0]+20,nzero[:,1]-174]=80

#rotate 3
M = cv2.getRotationMatrix2D((485/2,485/2),45,1)
img3 = cv2.warpAffine(img3,M,(485,485))

#get nonzero for 3, and move it...
nzero=np.row_stack(np.where(img3>0)).T
img3[nzero[:,0],nzero[:,1]]=0
img3[nzero[:,0]+100,nzero[:,1]+145]=80

#add them...
img = img1+img2+img3

plt.imshow(img,cmap='Greys',interpolation='none')
ax=plt.gca()
ax.vlines(227,0,450,color='grey')
ax.text(50,50,"Fake Image",fontsize=20)
fig=plt.gcf()
fig.set_size_inches(20,12)
plt.show()

In [ ]:
iom = larcv.IOManager(larcv.IOManager.kWRITE)
iom.set_verbosity(0)
iom.set_out_file("/tmp/aho.root")
iom.initialize()
fake_evimg=iom.get_data(larcv.kProductImage2D,"test")

img2d1 = larcv.as_image2d(img1)
img2d2 = larcv.as_image2d(img2)
img2d3 = larcv.as_image2d(img3)

imeta1 = larcv.ImageMeta(485,2910.0,
                   485,485,
                   166.0,5424.0,
                   0)

imeta2 = larcv.ImageMeta(485,2910.0,
                   485,485,
                   621.0,5424.0,
                   1)

imeta3 = larcv.ImageMeta(485,2910.0,
                   485,485,
                   343.,5424.0,
                   2)

img2d1 = larcv.Image2D(imeta1,img2d1.as_vector())
img2d2 = larcv.Image2D(imeta2,img2d2.as_vector())
img2d3 = larcv.Image2D(imeta3,img2d3.as_vector())


fake_evimg.Append(img2d1)
fake_evimg.Append(img2d2)
fake_evimg.Append(img2d3)

iom.set_id(0,0,1)
iom.save_entry()
iom.finalize()

In [ ]:
proc = larcv.ProcessDriver("ProcessDriver")
proc.configure('track_vertex_estimator_unit.fcl')
f=ROOT.std.string("/tmp/aho.root")
flist=ROOT.std.vector('string')()
flist.push_back(f)
proc.override_input_file(flist)
proc.initialize()
larbysimg = proc.process_ptr(0)

In [ ]:
proc.batch_process(0,1)

In [ ]:
mgr=larbysimg.Manager()
img_v = []
pygeo = geo2d.PyDraw()

In [ ]:
plane=0
for mat in mgr.InputImages():
    img_v.append(pygeo.image(mat))
    shape_img=np.where(img_v[plane]>0.0,1.0,0.0).astype(np.uint8)
    fig,ax=plt.subplots(figsize=(12,12),facecolor='w')
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    plt.xlabel('Time [6 ticks]',fontsize=20)
    plt.ylabel('Wire [2 wires]',fontsize=20)
    plt.tick_params(labelsize=20)
    ax.set_aspect(0.8)
    plt.tight_layout()
    plt.show()
    plane+=1

In [ ]:
colors=['red','green','blue','orange','magenta','cyan','pink']
colors*=10

dm=mgr.DataManager()
data=dm.Data(0)

for plane in xrange(3):
    ix=0
    fig,ax=plt.subplots(figsize=(12,12),facecolor='w')
    print 'Plane',plane
    shape_img = img_v[plane]
    shape_img=np.where(img_v[plane]>0.0,1.0,0.0).astype(np.uint8)
    plt.imshow(shape_img,cmap='Greys',interpolation='none')

    nz_pixels=np.where(shape_img>0.0)

    vertex_seeds = data.harvest_seeds(plane)
    print "Plane ",plane," w/ ",vertex_seeds
    cluscomp_v = vertex_seeds.get_compounds()
    
    #plot the atomics
    for cluscomp_id in xrange(cluscomp_v.size()):
        cluscomp = cluscomp_v[cluscomp_id]
        for atomic_id in xrange(cluscomp.size()):
            atomic = cluscomp[atomic_id]
            pts=[[atomic[p_id].x,atomic[p_id].y] for p_id in xrange(atomic.size())]
            if len(pts)>0: 
                pts.append(pts[0])
                pts=np.array(pts)
                plt.plot(pts[:,0],pts[:,1],'-o',lw=5,color=colors[ix])
                ix+=1

    
    points = data.harvest_seed_points(plane)
    print "Found number of seeds ",points.size()
    pts_v = np.array([[points[i].x,points[i].y] for i in xrange(points.size())])
    if pts_v.size>0:
        plt.plot(pts_v[:,0],pts_v[:,1],'*',markersize=30,color='yellow')

    ax.set_aspect(1.0)
    #plt.tight_layout()
    ax.set_ylim(np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
    ax.set_xlim(np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)
    plt.xlabel('Time [6 ticks]',fontsize=20)
    plt.ylabel('Wire [2 wires]',fontsize=20)
    plt.tick_params(labelsize=20)
    ax.set_aspect(0.8)
    #ax.text(150,300,"Yellow == Defect Seed\nOrange == PCA Seed",fontsize=30,fontweight='bold')
    plt.show()

In [ ]:
print mgr.GetClusterAlg(1)

In [ ]:
import matplotlib.patches as patches
alg=mgr.GetClusterAlg(1).Algo()

dm=mgr.DataManager()
ref_data = dm.Data(1)
tickscore0_y=[]
tickscore0_x=[]

score0_v = alg.TimeBinnedScore0Mean()
for idx in xrange(score0_v.size()):
    v = score0_v[idx]
    tickscore0_y.append(v)
    tickscore0_x.append(idx*1 + alg.TimeBinMin())
        
tickscore1_y=[]
tickscore1_x=[]
score1_v = alg.TimeBinnedScore1Mean()
for idx in xrange(score1_v.size()):
    v = score1_v[idx]
    tickscore1_y.append(v)
    tickscore1_x.append(idx*1 + alg.TimeBinMin())

tickscore0_x = np.array(tickscore0_x)
tickscore0_y = np.array(tickscore0_y)
tickscore1_x = np.array(tickscore1_x)
tickscore1_y = np.array(tickscore1_y)
ymin = tickscore0_y.min()
ymax = tickscore0_y.max()
if ymin > tickscore1_y.min(): ymin = tickscore1_y.min()
if ymax < tickscore1_y.max(): ymax = tickscore1_y.max()
    
fig,ax = plt.subplots(figsize=(16,8),facecolor='w')
plt.plot(tickscore0_x,tickscore0_y,marker='o',linestyle='-',color='red',markersize=10)
plt.plot(tickscore1_x,tickscore1_y,marker='o',linestyle='--',
         markeredgewidth=1,markeredgecolor='blue',markerfacecolor='None',markersize=10)


minimum_v  = alg.TimeBinnedScoreMinIndex()
minrange_v = alg.TimeBinnedScoreMinRange()
for idx in xrange(minimum_v.size()):
    xval = tickscore0_x[minimum_v[idx]]
    plt.plot([xval,xval],[0,360],marker='',linestyle='--',color='black',linewidth=2)
    xstart = tickscore0_x[minrange_v[idx].first]
    xend   = tickscore0_x[minrange_v[idx].second]
    ax.axvspan(xstart,xend, alpha=0.3, color='orange')

plt.xlabel('Time [6 ticks]',fontsize=20,fontweight='bold')
plt.ylabel('Summed Angle Difference [deg.]',fontsize=20,fontweight='bold')
plt.tick_params(labelsize=20)
plt.tight_layout()
ax=plt.gca()
ax.set_ylim(-1,ymax*1.1)
ax.set_xlim(tickscore0_x.min(),tickscore0_x.max())
plt.grid()
#ax.set_xlim(450,480)
plt.show()

In [ ]:
import matplotlib.patches as patches
alg=mgr.GetClusterAlg(1).Algo()

dm=mgr.DataManager()
ref_data = dm.Data(1)
for plane in xrange(3):
    wirescore_y = []
    wirescore_x = []
    score_v = alg.WireBinnedScoreMean(plane)
    for idx in xrange(score_v.size()):
        wirescore_y.append(score_v[idx])
        wirescore_x.append(idx + alg.WireBinMin(plane))

    wirescore_y = np.array(wirescore_y)
    wirescore_x = np.array(wirescore_x)
    
    
    xmin=0
    xmax=1
    ymin=0
    ymax=1
    
    if wirescore_y.size > 0 :
        ymin = wirescore_y.min()
        ymax = wirescore_y.max()

    if wirescore_x.size > 0 :
        xmin = wirescore_x.min()
        xmax = wirescore_x.max()

    fig,ax = plt.subplots(figsize=(16,8),facecolor='w')
    plt.plot(wirescore_x,wirescore_y,marker='o',linestyle='-',color='blue',markersize=10)


    minimum_v  = alg.WireBinnedScoreMinIndex(plane)
    minrange_v = alg.WireBinnedScoreMinRange(plane)
    for idx in xrange(minimum_v.size()):
        xval = wirescore_x[minimum_v[idx]]
        plt.plot([xval,xval],[0,360],marker='',linestyle='--',color='black',linewidth=2)
        xstart = wirescore_x[minrange_v[idx].first]
        xend   = wirescore_x[minrange_v[idx].second]
        ax.axvspan(xstart,xend, alpha=0.3, color='orange')

    plt.xlabel('Wire [2 wires]',fontsize=20,fontweight='bold')
    plt.ylabel('Summed Angle Difference [deg.]',fontsize=20,fontweight='bold')
    plt.tick_params(labelsize=20)
    plt.tight_layout()
    ax=plt.gca()
    ax.set_ylim(-1,ymax*1.1)
    ax.set_xlim(xmin,xmax)
    plt.grid()
    #ax.set_xlim(450,480)
    plt.show()

In [ ]:
dm.Data(0).harvest_seeds(0)

In [ ]:
import matplotlib.patches as patches

for plane in xrange(3):
    fig,ax=plt.subplots(figsize=(12,12),facecolor='w')
    shape_img=np.where(img_v[plane]>10.0,1.0,0.0).astype(np.uint8)
    plt.imshow(shape_img,cmap='Greys',interpolation='none')
    nz_pixels=np.where(shape_img>0.0)

    alg = mgr.GetClusterAlg(1).Algo()
    
    cv=larbysimg.Manager().GetClusterAlg(1)
    mgr=larbysimg.Manager()
    dm=mgr.DataManager()
    
    #Getting seeds
    seeds_data=dm.Data(0)
    seeds = seeds_data.harvest_seeds(plane)
    for compound in seeds.get_compounds():
        for defect in compound.get_defects():
            pt = defect._pt_defect
            plt.plot([pt.x],[pt.y],marker='o',color='blue',markersize=10)
    for pt in seeds.get_pcaxs():
        plt.plot([pt.x],[pt.y],marker='o',color='red',markersize=10)
            
    ref_data   = dm.Data(1)
    plane_scan = alg.PlaneInfo(plane)

    for circle in plane_scan._circle_scan_v:
        c=patches.Circle((circle.center.x,circle.center.y),circle.radius,ec='cyan',alpha=0.05,fc='none',lw=10)
        ax.add_patch(c)

    minimum_v = alg.TimeBinnedScoreMinIndex()
    for idx in xrange(minimum_v.size()):
        xval = minimum_v[idx] + alg.TimeBinMin()
        plt.plot([xval,xval],[0,2000],marker='',linestyle='--',color='magenta',linewidth=2)

    vtx_vv = ref_data.get_circle_vertex()
    for vtx_idx in xrange(len(vtx_vv)):
        vtx = vtx_vv[vtx_idx][plane]
        color='magenta'
        if ref_data.get_type(vtx_idx) == 1:
            color='yellow'
        plt.plot([vtx.center.x],[vtx.center.y],marker='$\star$',color=color,markersize=24)

    ymin,ymax = (np.min(nz_pixels[0])-10,np.max(nz_pixels[0])+10)
    xmin,xmax = (np.min(nz_pixels[1])-10,np.max(nz_pixels[1])+10)

    ax.set_ylim(ymin,ymax)
    ax.set_xlim(xmin,xmax)
    plt.xlabel('Time [6 ticks]',fontsize=20)
    plt.ylabel('Wire [2 wires]',fontsize=20)
    plt.tick_params(labelsize=20)
    ax.set_aspect(0.8)
    plt.tight_layout()
    ax=plt.gca()
    plt.show()
    
